In [1]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date


In [2]:
PATH_IN = '../data_jeannie/raw/'
PATH_OUT = '../data_jeannie/per_week/'

In [3]:
PATH_IN = '../data/raw/'
PATH_OUT = '../data/per_week/'

In [4]:
def get_pdetail(hmtl_data):
    player_details = dict()
    soup = BeautifulSoup(hmtl_data)
    player_details_html = soup.find_all("div", {"class": "histogram horiz"})
    for info in player_details_html:
        player_details[info['label']] = info['value']
    return player_details

In [5]:
def process_players(file_name):

    # load data
    players = []
    path_player_file = PATH_IN +'/'+ file_name+'.csv'
    with open(path_player_file,'r',encoding='utf-8') as f:
        player_reader = csv.reader(f, delimiter=',', quotechar='"')

        for row in player_reader:
            players_detail = dict()

            players_detail['id'] = row[0]

            players_detail['role'] = row[1]
            players_detail['name'] = row[2]
            players_detail['club'] = row[3][row[3].find('clubs/')+6:row[3].find('/logo.png')]
            players_detail['etranger'] = 'OUI' if '/ch.png' not in row[5] else 'NON' 

            if '<div' in row[6]:
                players_detail.update(get_pdetail(row[6])) 
            players.append(players_detail)

    # Save to excel
    today = date.today()
    df = pd.DataFrame(players).set_index('id')
    path_dir_out = PATH_OUT +f'/{str(today)}/'
    file_name_out =  file_name+'.xlsx'

    os.makedirs(path_dir_out,exist_ok=True)
    df.to_excel(path_dir_out+'/'+file_name_out)

In [6]:
process_players('my_players')

In [7]:
process_players('other_players')